<a href="https://colab.research.google.com/github/rexxar280903/Qml/blob/main/PennyLane/svm_classifcation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch
from torch.nn.functional import relu
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
import matplotlib.pyplot as plt
import pandas as pd
np.random.seed(42)
from sklearn.decomposition import PCA
from tqdm import tqdm
import numpy as np



In [6]:

X_train = np.load(r"C:\Users\user\Videos\belajar\Qml\PennyLane\ekstrak\demonstration_assets_2q_train_images_600.npy")
y_train = np.load(r"C:\Users\user\Videos\belajar\Qml\PennyLane\ekstrak\demonstration_assets_2q_train_labels_600.npy")
X_test = np.load(r"C:\Users\user\Videos\belajar\Qml\PennyLane\ekstrak\demonstration_assets_2q_test_images.npy_100.npy")
y_test = np.load(r"C:\Users\user\Videos\belajar\Qml\PennyLane\ekstrak\demonstration_assets_2q_test_labels.npy_100.npy")


In [7]:
X_train

array([[[[ 1.        , -0.2424635 , -0.04614222, -0.2424635 ],
         [ 1.        , -0.2424635 , -0.04614222, -0.2424635 ],
         [ 1.        , -0.2424635 , -0.04614222, -0.2424635 ],
         ...,
         [ 1.        , -0.2424635 , -0.04614222, -0.2424635 ],
         [ 1.        , -0.2424635 , -0.04614222, -0.2424635 ],
         [ 1.        , -0.2424635 , -0.04614222, -0.2424635 ]],

        [[ 1.        , -0.2424635 , -0.04614222, -0.2424635 ],
         [ 1.        , -0.2424635 , -0.04614222, -0.2424635 ],
         [ 1.        , -0.2424635 , -0.04614222, -0.2424635 ],
         ...,
         [ 1.        , -0.2424635 , -0.04614222, -0.2424635 ],
         [ 1.        , -0.2424635 , -0.04614222, -0.24244509],
         [ 1.        , -0.2424635 , -0.037176  , -0.2424635 ]],

        [[ 1.        , -0.2424635 , -0.04614222, -0.2424635 ],
         [ 1.        , -0.2424635 , -0.04614222, -0.2424635 ],
         [ 1.        , -0.2424635 , -0.04614222, -0.2424635 ],
         ...,
         

In [9]:
print(X_train.shape)


(600, 14, 14, 4)


In [10]:
print(np.isnan(X_train).sum())  # Cek ada NaN atau tidak
print(type(X_train))  # Pastikan X_train berupa numpy array


0
<class 'numpy.ndarray'>


In [11]:
# Ubah data menjadi 2D jika masih dalam bentuk 3D atau 4D
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Pastikan tidak ada NaN
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

# Sesuaikan jumlah komponen PCA dengan jumlah fitur yang tersedia
n_components = min(4, X_train.shape[1])  

# Lakukan PCA
pca = PCA(n_components=n_components)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Define a quantum device
n_qubits = n_components  # Use reduced number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum embedding
@qml.qnode(dev)
def quantum_embedding(x):
    qml.AngleEmbedding(x, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights=np.ones((1, n_qubits)), wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))

# Compute the quantum kernel with progress tracking
def quantum_kernel(X1, X2):
    kernel_matrix = np.zeros((len(X1), len(X2)))
    for i in tqdm(range(len(X1)), desc="Computing Quantum Kernel"):
        for j in range(len(X2)):
            kernel_matrix[i, j] = np.dot(quantum_embedding(X1[i]), quantum_embedding(X2[j]))
    return kernel_matrix

# Compute the kernel matrix
kernel_matrix = quantum_kernel(X_train, X_train)

# Train the classical SVM with the quantum kernel
clf = SVC(kernel="precomputed")
clf.fit(kernel_matrix, y_train)

# Prediction function with progress tracking
def predict(X_test):
    kernel_test = np.zeros((len(X_test), len(X_train)))
    for i in tqdm(range(len(X_test)), desc="Computing Test Kernel"):
        for j in range(len(X_train)):
            kernel_test[i, j] = np.dot(quantum_embedding(X_test[i]), quantum_embedding(X_train[j]))
    return clf.predict(kernel_test)

# Test the classifier
predictions = predict(X_test)
accuracy = np.mean(predictions == y_test)
print("Accuracy:", accuracy)


Computing Quantum Kernel:   0%|          | 0/600 [00:00<?, ?it/s]

Computing Test Kernel: 100%|██████████| 100/100 [02:45<00:00,  1.66s/it]

Accuracy: 0.31


In [12]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\user\Videos\belajar\Qml\CNN\Hugging Face\hug\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
# Simpan model SVM
joblib.dump(clf, "quantum_svm_model.pkl")

# Simpan PCA jika ingin digunakan kembali
joblib.dump(pca, "pca_model.pkl")

Model berhasil disimpan!
